# 🕵🏻‍♀️ Create a RAG system expert in a GitHub repository and log your predictions in Argilla

In this tutorial, we'll show you how to create a RAG system that can answer questions about a specific GitHub repository. As example, we will target the [Argilla repository](https://github.com/argilla-io/argilla). This RAG system will target the docs of the repository, as that's where most of the natural language information about the repository can be found.

This tutorial includes the following steps:

-   Setting up the Argilla callback handler for LlamaIndex.
-   Initializing a GitHub client
-   Creating an index with a specific set of files from the GitHub repository of our choice.
-   Create a RAG system out of the Argilla repository, ask questions, and automatically log the answers to Argilla.

This tutorial is based on the [Github Repository Reader](https://docs.llamaindex.ai/en/stable/examples/data_connectors/GithubRepositoryReaderDemo/) made by LlamaIndex.

## Getting started

### Deploy the Argilla server¶

If you already have deployed Argilla, you can skip this step. Otherwise, you can quickly deploy Argilla following [this guide](https://docs.argilla.io/latest/getting_started/quickstart/).

### Set up the environment¶

To complete this tutorial, you need to install this integration and a third-party library via pip.

!!! note
    Check the integration GitHub repository [here](https://github.com/argilla-io/argilla-llama-index).

In [ ]:
!pip install "argilla-llama-index"
!pip install "llama-index-readers-github==0.1.9"

Let's make the required imports:

In [1]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    set_global_handler,
)
from llama_index.llms.openai import OpenAI
from llama_index.readers.github import (
    GithubClient,
    GithubRepositoryReader,
)

We need to set the OpenAI API key and the GitHub token. The OpenAI API key is required to run queries using GPT models, while the GitHub token ensures you have access to the repository you're using. Although the GitHub token might not be necessary for public repositories, it is still recommended.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["GITHUB_TOKEN"] = "ghp_..."
github_token = os.getenv("GITHUB_TOKEN")

## Set the Argilla's LlamaIndex handler

To easily log your data into Argilla within your LlamaIndex workflow, you only need a simple step. Just call the Argilla global handler for Llama Index before starting production with your LLM. This ensured that the predictions obtained using Llama Index are automatically logged to the Argilla instance.

- `dataset_name`: The name of the dataset. If the dataset does not exist, it will be created with the specified name. Otherwise, it will be updated.
- `api_url`: The URL to connect to the Argilla instance.
- `api_key`: The API key to authenticate with the Argilla instance.
- `number_of_retrievals`: The number of retrieved documents to be logged. Defaults to 0.
- `workspace_name`: The name of the workspace to log the data. By default, the first available workspace.

> For more information about the credentials, check the documentation for [users](https://docs.argilla.io/latest/how_to_guides/user/) and [workspaces](https://docs.argilla.io/latest/how_to_guides/workspace/).

In [ ]:
set_global_handler(
    "argilla",
    dataset_name="github_query_model",
    api_url="http://localhost:6900",
    api_key="argilla.apikey",
    number_of_retrievals=2,
)

## Retrieve the data from GitHub

First, we need to initialize the GitHub client, which will include the GitHub token for repository access.

In [4]:
github_client = GithubClient(github_token=github_token, verbose=True)

Before creating our `GithubRepositoryReader` instance, we need to adjust the nesting. Since the Jupyter kernel operates on an event loop, we must prevent this loop from finishing before the repository is fully read.

In [5]:
import nest_asyncio

nest_asyncio.apply()

Now, let’s create a GithubRepositoryReader instance with the necessary repository details. In this case, we'll target the `main` branch of the `argilla` repository. As we will focus on the documentation, we will focus on the `argilla/docs/` folder, excluding images, json files, and ipynb files.

In [6]:
documents = GithubRepositoryReader(
    github_client=github_client,
    owner="argilla-io",
    repo="argilla",
    use_parser=False,
    verbose=False,
    filter_directories=(
        ["argilla/docs/"],
        GithubRepositoryReader.FilterType.INCLUDE,
    ),
    filter_file_extensions=(
        [
            ".png",
            ".jpg",
            ".jpeg",
            ".gif",
            ".svg",
            ".ico",
            ".json",
            ".ipynb",   # Erase this line if you want to include notebooks

        ],
        GithubRepositoryReader.FilterType.EXCLUDE,
    ),
).load_data(branch="main")

## Create the index and make some queries

Now, let's create a LlamaIndex index out of this document, and we can start querying the RAG system.

In [7]:
# LLM settings
Settings.llm = OpenAI(
    model="gpt-3.5-turbo", temperature=0.8, openai_api_key=openai_api_key
)

# Load the data and create the index
index = VectorStoreIndex.from_documents(documents)

# Create the query engine
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("How do I create a Dataset in Argilla?")
response

The generated response will be automatically logged in our Argilla instance. Check it out! From Argilla you can quickly have a look at your predictions and annotate them, so you can combine both synthetic data and human feedback.

![Argilla UI](../../assets/images/community/integrations/llamaindex_rag_1.png)


Let's ask a couple of more questions to see the overall behavior of the RAG chatbot. Remember that the answers are automatically logged into your Argilla instance.

In [10]:
questions = [
    "How can I list the available datasets?",
    "Which are the user credentials?",
    "Can I use markdown in Argilla?",
    "Could you explain how to annotate datasets in Argilla?",
]

answers = []

for question in questions:
    answers.append(query_engine.query(question))

for question, answer in zip(questions, answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("----------------------------")

Question: How can I list the available datasets?
Answer: You can list all the datasets available in a workspace by utilizing the `datasets` attribute of the `Workspace` class. Additionally, you can determine the number of datasets in a workspace by using `len(workspace.datasets)`. To list the datasets, you can iterate over them and print out each dataset. Remember that dataset settings are not preloaded when listing datasets, and if you need to work with settings, you must load them explicitly for each dataset.
----------------------------
Question: Which are the user credentials?
Answer: The user credentials in Argilla consist of a username, password, and API key.
----------------------------
Question: Can I use markdown in Argilla?
Answer: Yes, you can use Markdown in Argilla.
----------------------------
Question: Could you explain how to annotate datasets in Argilla?
Answer: To annotate datasets in Argilla, users can manage their data annotation projects by setting up `Users`, `Wor